In [ ]:
!pip install keras==2.3.1
!pip install tensorflow==2.1.0
!pip install keras-applications==1.0.7
!pip install image-classifiers==1.0.0
!pip install efficientnet==1.0.0

In [ ]:
!pip install segmentation-models

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline 
# 노트북에서 plot 바로 가능하게
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
IMAGE_PATH = '/content/drive/MyDrive/ML/Lung Cancer Segmentation/2d_images/'
MASK_PATH = '/content/drive/MyDrive/ML/Lung Cancer Segmentation/2d_masks/'
IMG_HEIGHT, IMG_WIDTH = 224, 224
SEED = 42

In [ ]:

all_images = [x for x in sorted(os.listdir(IMAGE_PATH)) if x[-4:] == '.tif']

x_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
print(x_data.shape)

for i, name in enumerate(all_images):
    im = cv2.imread(IMAGE_PATH + name, cv2.IMREAD_UNCHANGED).astype("int16").astype('float32') # unchanged -> alpha 채널까지 읽어드림
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
    im = np.clip(im, -600-750, -600+750) # CT windowing
    im = (im-np.min(im)) / (np.max(im) - np.min(im)) * 255# data normalization
    print('x ', i)
    x_data[i] = im
    
y_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(all_images):
    im = cv2.imread(MASK_PATH + name, cv2.IMREAD_UNCHANGED).astype('float32')/255.
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)
    print('y ', i)
    y_data[i] = im


In [ ]:
'''
x_data = x_data[:,:,:,np.newaxis]
x_data = np.concatenate((x_data, x_data, x_data), -1)
y_data = y_data[:,:,:,np.newaxis]
y_data = np.concatenate((y_data, y_data, y_data), -1)
print(y_data.shape)
print(x_data[0])
'''

In [ ]:
'''
x_data = np.load('/content/drive/MyDrive/ML/Lung Cancer Segmentation/lung_img_data.npy')
y_data = np.load('/content/drive/MyDrive/ML/Lung Cancer Segmentation/lung_mask_data.npy')
'''

In [ ]:
x_data = x_data[:,:,:,np.newaxis]
y_data = y_data[:,:,:,np.newaxis]

In [ ]:
import tensorflow as tf
x_data = tf.convert_to_tensor(x_data, dtype = 'float32')
x_data = tf.image.grayscale_to_rgb(x_data)
x_data = x_data.numpy()

In [ ]:
print(x_data.shape)
print(y_data.shape)
x_train = x_data[:133]
x_val = x_data[133:]
y_train = y_data[:133]
y_val = y_data[133:]
print(len(x_train))
print(len(x_val))

In [ ]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

In [ ]:
BACKBONE = 'vgg19'
preprocess_input = get_preprocessing(BACKBONE)

In [ ]:
# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

N = x_train.shape[-1]

# define model
model = Unet(backbone_name=BACKBONE, encoder_weights=None, input_shape=(None, None, N))
model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])

In [ ]:
print('x train: {}, y_train: {} \n x_val: {}, y_val: {}'.format(x_train.shape, y_train.shape, x_val.shape, y_val.shape))

In [ ]:
def my_generator(x_train, y_train, batch_size):
    data_generator = ImageDataGenerator(
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        rotation_range=10,
                        zoom_range=0.1).flow(x_train, x_train, batch_size, seed=SEED)
    mask_generator = ImageDataGenerator(
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        rotation_range=10,
                        zoom_range=0.1).flow(y_train, y_train, batch_size, seed=SEED)
    while True:
        x_batch, _ = data_generator.next()
        y_batch, _ = mask_generator.next()
        yield x_batch, y_batch

In [ ]:
image_batch, mask_batch = next(my_generator(x_train, y_train, 8))

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-lung unet '+BACKBONE+'.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
history = model.fit_generator(my_generator(x_train, y_train, 8),
                           steps_per_epoch = 20,
                           validation_data = (x_val, y_val),
                           epochs=100, verbose=1,
                           callbacks=callbacks)

In [ ]:
def plot_history(history, title):
    plt.figure(figsize=(10,3))
    # Plot training & validation accuracy values
    plt.subplot(121)
    plt.plot(history.history['iou_score'])
    plt.plot(history.history['val_iou_score'])
    plt.title('Model iou_score')
    plt.ylabel('iou_score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
plot_history(history, 'Unet - '+BACKBONE)

In [ ]:
model.load_weights('model-lung unet '+BACKBONE+'.h5')

In [ ]:
y_predict = model.predict(x_val)

for i in range(10):
    fig, ax = plt.subplots(1,3,figsize=(12,6))
    ax[0].set_title('Original')
    ax[1].set_title('Result')
    ax[2].set_title('Predicted Result')
    ax[0].imshow(x_val[i,:,:,0], cmap='gray')
    ax[1].imshow(y_val[i,:,:,0])
    ax[2].imshow(y_predict[i,:,:,0])

In [ ]:
print(y_predict[0])

In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(60,120))
for i in range(10):
  ax[i].imshow(y_predict[i,:,:,0])



In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(60,120))
for i in range(10):
  ax[i].imshow(y_val[i,:,:,0])

In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(60,120))
for i in range(10):
  ax[i].imshow(x_val[i,:,:,0], cmap='gray')